In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import duckdb
from uuid import uuid4
conn = duckdb.connect()

In [13]:
query = """
CREATE TABLE IF NOT EXISTS conversation (
    convo_id VARCHAR(36) PRIMARY KEY,
    content TEXT NOT NULL,
    role VARCHAR(10) CHECK (role IN ('user', 'assistant')),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    session_id VARCHAR(36)
);
"""
conn.execute(query).fetchall()

[]

In [14]:
conn.execute("SHOW TABLES;").df()

,name
0,conversation


In [15]:
query = """
INSERT INTO conversation (convo_id, content, role) VALUES (?, ?, ?);
"""

conn.execute(query, [str(uuid4()), "Hello, world!", "user"]).fetchall()

[(1,)]

In [16]:
query = """
SELECT * FROM conversation;
"""
conn.execute(query).df()

,convo_id,content,role,created_at,updated_at,session_id
0,d598b3ed-564e-4050-877f-67d15299f8f9,"Hello, world!",user,2025-11-11 22:05:54.905936,2025-11-11 22:05:54.905936,None


In [17]:
convo_id = conn.execute("SELECT convo_id FROM conversation LIMIT 1;").fetchone()[0]
query = """
UPDATE conversation
SET content = ?,
updated_at = CURRENT_TIMESTAMP
WHERE convo_id = ?;
"""

conn.execute(query, ["Holla world!", convo_id]).fetchall()

[(1,)]

In [18]:
query = """
SELECT * FROM conversation;
"""
conn.execute(query).df()

,convo_id,content,role,created_at,updated_at,session_id
0,d598b3ed-564e-4050-877f-67d15299f8f9,Holla world!,user,2025-11-11 22:05:54.905936,2025-11-11 22:05:54.938240,None


In [19]:
query = """
DELETE FROM conversation WHERE convo_id = ?;
"""

conn.execute(query, [convo_id]).fetchall()

[(1,)]

In [20]:
query = """
SELECT * FROM conversation;
"""
conn.execute(query).df()

,convo_id,content,role,created_at,updated_at,session_id


In [23]:
from uuid import uuid4
from datetime import datetime
from pydantic import BaseModel, Field
from typing import List, Optional, Any, Optional, Dict, Literal
from enum import StrEnum, IntEnum, Enum

class ConversationRole(StrEnum):
    USER = "user"
    ASSISTANT = "assistant"

class Artifact(BaseModel):
    type:Literal["sql", "data", "html"]
    data:Any

class Conversation(BaseModel):
    conversation_id:str = Field(default_factory=lambda: str(uuid4()))
    content:str
    role:ConversationRole
    created_at:datetime = Field(default_factory=datetime.now)
    updated_at:datetime = Field(default_factory=datetime.now)
    artifacts:Optional[List[Artifact]] = Field(default=None)
    session_id:Optional[str] = Field(default=None)

Conversation(content="Hi", role=ConversationRole.USER)

Conversation(conversation_id='c2488c19-1eb3-442c-a1da-778b1d91837a', content='Hi', role=<ConversationRole.USER: 'user'>, created_at=datetime.datetime(2025, 11, 11, 22, 34, 35, 99163), updated_at=datetime.datetime(2025, 11, 11, 22, 34, 35, 99163), artifacts=None, session_id=None)

In [ ]:
class Config:
    DATABASE_URL = "sqlite:///app.db"
    TIMEOUT = 30
    DEBUG = False

Config.DATABASE_URL

'sqlite:///app.db'

In [2]:
Config.TIMEOUT == 30

True

In [4]:
from enum import Enum

class Status(Enum):
    PENDING = "pending" 
    COMPLETED = "completed"

# These are PREVENTED:
# Status.PENDING = "oops"  # Error!
# Status.INVALID = "hack"  # Error!

# Built-in validation:
Status("pending")  # Returns Status.PENDING
# Status("invalid")  # Raises ValueError

# Iteration works:
for status in Status:
    print(status)  # All valid values


Status.PENDING
Status.COMPLETED


In [8]:
Status("pending")

<Status.PENDING: 'pending'>

In [9]:
# Fixed values but not choices
from types import MappingProxyType

class Config:
    _values = {
        "max_connections": 100,
        "timeout_seconds": 30,
        "api_key": "secret123"
    }
    
    # Immutable view
    VALUES = MappingProxyType(_values)

# User reads: Config.VALUES["timeout_seconds"]
# Config.VALUES["timeout_seconds"] = 50  # Error!


In [13]:
from enum import Enum

class DatabaseType(Enum):
    SQLITE = "sqlite"
    POSTGRES = "postgresql"
    MYSQL = "mysql"

class LogLevel(Enum):
    DEBUG = "debug"
    INFO = "info"
    ERROR = "error"

# Simple validation function
def validate_enum(value, enum_class):
    try:
        return enum_class(value)
    except ValueError:
        raise ValueError(f"Invalid {enum_class.__name__}: {value}")

# Usage
db_type = validate_enum("sqlite", DatabaseType)  # Returns DatabaseType.SQLITE
log_level = validate_enum("info", LogLevel)      # Returns LogLevel.INFO


In [18]:
db_type.value

'sqlite'

In [17]:
log_level.value

'info'

False

In [ ]:
from pydantic import BaseModel, Field
from enum import StrEnum

class Config(StrEnum):
    TEST = "test"

class ConfigModel(BaseModel):
    config:Config

# ConfigModel(config=Config.TEST)
# ConfigModel(config="test")

# The two above gives the same result, but why?

# ConfigModel(config=Config.TEST).model_dump() <- this return python dictionary of {'config': <Config.TEST: 'test'>}
# ConfigModel(config=Config.TEST).model_dump(mode='json') <- this return python dictionary of {'config': 'test'}
# ConfigModel(config=Config.TEST).model_dump_json() <- this return json string of "{'config': 'test'}"

In [30]:
def mock_func(config: ConfigModel):
    """
    Process configuration and print JSON output.
    
    Args:
        config (ConfigModel): Configuration model. The config field accepts 
            Config enum values ("test", "prod", "dev").
    """
    print(config.model_dump(mode='json'))

In [31]:
mock_func(ConfigModel(config=Config.TEST))

{'config': 'test'}
